In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, log_loss, r2_score
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.svm import SVC, SVR
from sklearn.linear_model import ElasticNet, LinearRegression
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaseEnsemble, VotingClassifier, GradientBoostingClassifier, RandomForestClassifier, StackingClassifier, StackingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('train_v9rqX0R.csv')
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
train.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [35]:
from sklearn.impute import SimpleImputer


si = SimpleImputer(strategy='median')
si1 = SimpleImputer(strategy='most_frequent')
le = LabelEncoder()

ct =  make_column_transformer((si1,make_column_selector(dtype_include=object)),
                              (si,make_column_selector(dtype_include=['int64',"float64"])),
                               remainder='passthrough').set_output(transform="pandas")
t_train = ct.fit_transform(train)

In [36]:
t_train.shape

(8523, 12)

In [37]:
t_train

,simpleimputer-1__Item_Identifier,simpleimputer-1__Item_Fat_Content,simpleimputer-1__Item_Type,simpleimputer-1__Outlet_Identifier,simpleimputer-1__Outlet_Size,simpleimputer-1__Outlet_Location_Type,simpleimputer-1__Outlet_Type,simpleimputer-2__Item_Weight,simpleimputer-2__Item_Visibility,simpleimputer-2__Item_MRP,simpleimputer-2__Outlet_Establishment_Year,simpleimputer-2__Item_Outlet_Sales
0,FDA15,Low Fat,Dairy,OUT049,Medium,Tier 1,Supermarket Type1,9.300,0.016047,249.8092,1999.0,3735.1380
1,DRC01,Regular,Soft Drinks,OUT018,Medium,Tier 3,Supermarket Type2,5.920,0.019278,48.2692,2009.0,443.4228
2,FDN15,Low Fat,Meat,OUT049,Medium,Tier 1,Supermarket Type1,17.500,0.016760,141.6180,1999.0,2097.2700
3,FDX07,Regular,Fruits and Vegetables,OUT010,Medium,Tier 3,Grocery Store,19.200,0.000000,182.0950,1998.0,732.3800
4,NCD19,Low Fat,Household,OUT013,High,Tier 3,Supermarket Type1,8.930,0.000000,53.8614,1987.0,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,Low Fat,Snack Foods,OUT013,High,Tier 3,Supermarket Type1,6.865,0.056783,214.5218,1987.0,2778.3834
8519,FDS36,Regular,Baking Goods,OUT045,Medium,Tier 2,Supermarket Type1,8.380,0.046982,108.1570,2002.0,549.2850
8520,NCJ29,Low Fat,Health and Hygiene,OUT035,Small,Tier 2,Supermarket Type1,10.600,0.035186,85.1224,2004.0,1193.1136
8521,FDN46,Regular,Snack Foods,OUT018,Medium,Tier 3,Supermarket Type2,7.210,0.145221,103.1332,2009.0,1845.5976


In [24]:
test = pd.read_csv('test_AbJTz2l.csv')
test.isna().sum()

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [25]:
ct =  make_column_transformer((si1,make_column_selector(dtype_include=object)),
                              (si,make_column_selector(dtype_include=['int64',"float64"])),remainder='passthrough').set_output(transform="pandas")
t_test = ct.fit_transform(test)

In [27]:
t_test.isna().sum()

simpleimputer-1__Item_Identifier              0
simpleimputer-1__Item_Fat_Content             0
simpleimputer-1__Item_Type                    0
simpleimputer-1__Outlet_Identifier            0
simpleimputer-1__Outlet_Size                  0
simpleimputer-1__Outlet_Location_Type         0
simpleimputer-1__Outlet_Type                  0
simpleimputer-2__Item_Weight                  0
simpleimputer-2__Item_Visibility              0
simpleimputer-2__Item_MRP                     0
simpleimputer-2__Outlet_Establishment_Year    0
dtype: int64

In [29]:
y_train = t_train['simpleimputer-2__Outlet_Establishment_Year']
X_train = t_train.drop('simpleimputer-2__Outlet_Establishment_Year', axis=1)

In [30]:
lr = LinearRegression()
xgboost = XGBRegressor(random_state=24)
ela = ElasticNet(random_state=24)
cat = CatBoostRegressor(random_state=24)
light = LGBMRegressor(random_state=24)

stack = StackingRegressor([('LR', lr), ('LIG', light), ('CAT', cat), ('ELA', ela)], final_estimator=light, passthrough=True)



In [31]:
stack.fit(X_train, y_train)

ValueError: could not convert string to float: 'FDA15'